# Contour plot with colorbar


In [ ]:
import numpy as np

import matplotlib as mpl
from mpllayout import geometry as geo, layout as mlay, solver as msol
from mpllayout.matplotlibutils import subplots

In [ ]:
# This ensures matplotlib figures don't clip any whitespace
%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

## Create a layout

In [ ]:
## Create the figure and two axes (one for the contour plot and one for the colorbar)

### Primitives
# Create a `Layout` to track all geometric primitives and constraints
layout = mlay.Layout()

# Create the main geometric primitives
layout.add_prim(geo.Point(), "Origin")
layout.add_prim(geo.Quadrilateral(), "Figure")
layout.add_prim(geo.Axes(), "AxesContour")
layout.add_prim(geo.Axes(), "AxesColorbar")

### Constraints

# Fix the origin
layout.add_constraint(geo.Fix((np.array([0, 0]),)), ("Origin",))

# Make all the quadrilaterals rectangular
layout.add_constraint(geo.Box(()), ("Figure",))
layout.add_constraint(geo.Box(()), ("AxesContour/Frame",))
layout.add_constraint(geo.Box(()), ("AxesColorbar/Frame",))

## Figure size
layout.add_constraint(geo.Length((7,)), ("Figure/Line0",))
layout.add_constraint(geo.Coincident(()), ("Figure/Line0/Point0", "Origin"))

## Axes sizes

# Set the aspect ratio to match physical dimensions
layout.add_constraint(
    geo.RelativeLength((2,)), ("AxesContour/Frame/Line0", "AxesContour/Frame/Line1")
)

# Set the color bar height to 1/16 inch
layout.add_constraint(geo.Length((1 / 16,)), ("AxesColorbar/Frame/Line1",))

## Align the color bar with the contour plot
layout.add_constraint(
    geo.Collinear(()), ("AxesContour/Frame/Line3", "AxesColorbar/Frame/Line3")
)
layout.add_constraint(
    geo.Collinear(()), ("AxesContour/Frame/Line1", "AxesColorbar/Frame/Line1")
)

## Margins
# Place the color bar 1/16 inch above the contour plot
layout.add_constraint(
    geo.YDistanceMidpoints((1 / 32,)),
    ("AxesContour/Frame/Line2", "AxesColorbar/Frame/Line0"),
)

# Set the left margin to 1/2 inch from the contour plot
layout.add_constraint(
    geo.XDistanceMidpoints((1 / 2,)), ("Figure/Line3", "AxesContour/Frame/Line3")
)

# Set the right margin 1/8 inch from the contour plot
layout.add_constraint(
    geo.XDistanceMidpoints((1 / 8,)), ("AxesContour/Frame/Line1", "Figure/Line1")
)

# Set the top margin to 1/2 inch
layout.add_constraint(
    geo.YDistanceMidpoints((1 / 2,)), ("AxesColorbar/Frame/Line2", "Figure/Line2")
)

# Set the bottom margin to 1/2 inch
layout.add_constraint(
    geo.YDistanceMidpoints((1 / 2,)), ("Figure/Line0", "AxesContour/Frame/Line0")
)

In [ ]:
## Solve the constrained system of plots

root_prim_n, solve_info = msol.solve(layout.root_prim, *layout.flat_constraints())

print(solve_info)

fig, axs = subplots(root_prim_n)
print(fig.get_size_inches())

x = np.linspace(0, 10, 51)
y = np.linspace(0, 5, 26)
xx, yy = np.meshgrid(x, y)
z = (xx - 5) ** 2 + (yy - 2.5) ** 2

cset = axs["AxesContour"].contourf(x, y, z)

axs["AxesContour"].set_xlabel("x [cm]")
axs["AxesContour"].set_ylabel("y [cm]")

fig.colorbar(cset, cax=axs["AxesColorbar"], orientation="horizontal")
axs["AxesColorbar"].xaxis.set_label_text("z [cm]")
axs["AxesColorbar"].xaxis.set_tick_params(
    top=True, labeltop=True, bottom=False, labelbottom=False
)
axs["AxesColorbar"].xaxis.set_label_position(position="top")

fig.savefig("out/contour_colorbar.png")

fig